In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heart-attack-prediction/data.csv


In [23]:
heart_attack_df = pd.read_csv("/kaggle/input/heart-attack-prediction/data.csv")

In [35]:

print("Shape of dataset:", heart_attack_df.shape)
print("First 20 rows:")
display(heart_attack_df.head(20))

Shape of dataset: (294, 14)
First 20 rows:


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,28,1,2,130,132,0,2,185,0,0.0,?,?,?,0
1,29,1,2,120,243,0,0,160,0,0.0,?,?,?,0
2,29,1,2,140,?,0,0,170,0,0.0,?,?,?,0
3,30,0,1,170,237,0,1,170,0,0.0,?,?,6,0
4,31,0,2,100,219,0,1,150,0,0.0,?,?,?,0
5,32,0,2,105,198,0,0,165,0,0.0,?,?,?,0
6,32,1,2,110,225,0,0,184,0,0.0,?,?,?,0
7,32,1,2,125,254,0,0,155,0,0.0,?,?,?,0
8,33,1,3,120,298,0,0,185,0,0.0,?,?,?,0
9,34,0,2,130,161,0,0,190,0,0.0,?,?,?,0


In [36]:
# Total rows in your dataset
total_rows = 294  

# Create a summary DataFrame
question_mark_summary = pd.DataFrame({
    "Column": heart_attack_df.columns,
    "QuestionMark_Count": [
        (heart_attack_df[col] == "?").sum() for col in heart_attack_df.columns
    ]
})

# Add percentage column
question_mark_summary["QuestionMark_Percent"] = (
    question_mark_summary["QuestionMark_Count"] / total_rows * 100
).round(2)

print(question_mark_summary)


        Column  QuestionMark_Count  QuestionMark_Percent
0          age                   0                  0.00
1          sex                   0                  0.00
2           cp                   0                  0.00
3     trestbps                   1                  0.34
4         chol                  23                  7.82
5          fbs                   8                  2.72
6      restecg                   1                  0.34
7      thalach                   1                  0.34
8        exang                   1                  0.34
9      oldpeak                   0                  0.00
10       slope                 190                 64.63
11          ca                 291                 98.98
12        thal                 266                 90.48
13  num                          0                  0.00


In [37]:
cols_to_drop = ["slope", "ca", "thal"]
heart_attack_df = heart_attack_df.drop(columns=cols_to_drop)

print("Remaining columns:")
print(heart_attack_df.columns)

Remaining columns:
Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'num       '],
      dtype='object')


3 age: age in years
4 sex: sex (1 = male; 0 = female)
9 cp: chest pain type
-- Value 1: typical angina
-- Value 2: atypical angina
-- Value 3: non-anginal pain
-- Value 4: asymptomatic
10 trestbps: resting blood pressure (in mm Hg on admission to the
hospital)
12 chol: serum cholestoral in mg/dl
16 fbs: (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
19 restecg: resting electrocardiographic results
-- Value 0: normal
-- Value 1: having ST-T wave abnormality (T wave inversions and/or ST
elevation or depression of > 0.05 mV)
-- Value 2: showing probable or definite left ventricular hypertrophy
32 thalach: maximum heart rate achieved
38 exang: exercise induced angina (1 = yes; 0 = no)
40 oldpeak = ST depression induced by exercise relative to rest
58 num: diagnosis of heart disease (angiographic disease status)
-- Value 0: < 50% diameter narrowing -- Value 1: > 50% diameter narrowing
(in any major vessel: attributes 59 through 68 are vessels)

In [38]:
# Strip whitespace from column names
heart_attack_df.columns = heart_attack_df.columns.str.strip()

print("Cleaned column names:")
print(list(heart_attack_df.columns))


Cleaned column names:
['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'num']


In [43]:
# Create new binary column: 0 = no heart attack, 1 = heart attack
heart_attack_df["heart_disease"] = heart_attack_df["num"].apply(
    lambda x: 1 if x > 0 else 0
)

# Features (X) and Target (y)
X = heart_attack_df.drop(columns=["heart_attack"])
y = heart_attack_df["heart_attack"]

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)



Shape of X: (294, 12)
Shape of y: (294,)


In [44]:
print(f" Target distribution:")
print(y.value_counts())
print(f"Baseline accuracy: {max(y.value_counts(normalize=True)):.3f}")

 Target distribution:
heart_attack
0    188
1    106
Name: count, dtype: int64
Baseline accuracy: 0.639
